In [1]:
import pandas as pd
import sqlite3
import json

write_conn = sqlite3.connect("../data/metrics_full_run.db")
write_conn.execute("PRAGMA foreign_keys = 1")
cur = write_conn.cursor()

In [2]:
cur.execute("CREATE TABLE IF NOT EXISTS context(context_id TEXT PRIMARY KEY, " \
            "context TEXT, " \
            "justice TEXT, " \
            "turn_text TEXT, " \
            "transcript_id TEXT, " \
            "FOREIGN KEY (transcript_id) REFERENCES transcript(transcript_id) ON DELETE CASCADE);")

cur.execute("CREATE TABLE IF NOT EXISTS transcript(transcript_id TEXT PRIMARY KEY, " \
            "case_facts TEXT, " \
            "legal_question TEXT, " \
            "conclusion TEXT);")

cur.execute("CREATE TABLE IF NOT EXISTS remark ( " \
        	"remark_id TEXT PRIMARY KEY, " \
        	"model TEXT, " \
        	"prompting_strategy TEXT," \
        	"justice TEXT, " \
        	"remark_text TEXT, " \
        	"log_id TEXT, " \
        	"context_id TEXT, " \
        	"FOREIGN KEY (context_id) REFERENCES context(context_id));")

cur.execute("CREATE TABLE IF NOT EXISTS distributional_metrics ( " \
        	"distributional_metric_id TEXT PRIMARY KEY, " \
        	"classification_model TEXT, " \
        	"metric_name TEXT, " \
        	"classification TEXT, " \
        	"remark_id TEXT, " \
        	"log_id TEXT, " \
        	"FOREIGN KEY (remark_id) REFERENCES remark(remark_id));")

cur.execute("CREATE TABLE IF NOT EXISTS comparative_metrics ( " \
        	"comparative_metric_id TEXT PRIMARY KEY, " \
        	"classification_model TEXT, " \
        	"metric_name TEXT, " \
        	"classification TEXT, " \
        	"remark1_id TEXT REFERENCES remark(remark_id), " \
            "remark2_id TEXT REFERENCES remark(remark_id), " \
        	"log_id TEXT);")

In [4]:
def restructure_context(context_as_string):
    context_as_list = json.loads(context_as_string)
    restructured_context_list = []
    for turn in context_as_list:
        if turn["speaker"]["role"] == "scotus_justice":
            role_text = turn["speaker"]["name"]
        else:
            role_text = turn["speaker"]["side"]
        restructured_turn = {"content": turn["text"], "role": role_text}
        restructured_context_list.append(restructured_turn)
    return str(restructured_context_list)

In [5]:
read_conn = sqlite3.connect("../data/raw_data/transcripts_raw.db")
transcripts_df = pd.read_sql_query("SELECT * from transcript WHERE CAST(SUBSTR(transcript_id, 1, 4) AS INTEGER) > 2023;", read_conn)
contexts_df = pd.read_sql_query("SELECT * from context WHERE CAST(SUBSTR(context_id, 1, 4) AS INTEGER) > 2023 AND turn_text LIKE '% % % %';", read_conn)

# reformat the context column
contexts_df["context"] = contexts_df["context"].apply(restructure_context)

In [11]:
transcripts_df.to_sql('transcript', write_conn, if_exists='append', index=False)
contexts_df.to_sql('context', write_conn, if_exists='append', index=False)

7262

In [12]:
def create_transcript_context_view(new_view_name, transcript_table, context_table):    
    create_view_sql = f"""
    CREATE VIEW IF NOT EXISTS {new_view_name} AS
    SELECT transcript.transcript_id as transcript_id,
        {transcript_table}.case_facts as case_facts,
        {transcript_table}.legal_question as legal_question,
        {context_table}.context_id as context_id,
        {context_table}.context as context,
        {context_table}.justice as justice,
        {context_table}.turn_text as turn_text
    FROM {transcript_table}
    JOIN {context_table} ON {transcript_table}.transcript_id = {context_table}.transcript_id;
    """

    cur.execute(create_view_sql)
    write_conn.commit()

create_transcript_context_view("transcript_and_context", "transcript", "context")

In [14]:
def create_remark_transcript_context_view(new_view_name, transcript_table, context_table, remark_table):
    create_view_sql = f"""CREATE VIEW IF NOT EXISTS {new_view_name} AS
    SELECT
        {remark_table}.remark_id as remark_id,
        {remark_table}.justice as justice,
        {remark_table}.remark_text as remark_text,
        {remark_table}.log_id as remark_log_id,
        {context_table}.context as context,
        {transcript_table}.case_facts as case_facts,
        {transcript_table}.legal_question as legal_question
    FROM remark
    JOIN context ON remark.context_id = context.context_id
    JOIN transcript ON context.transcript_id = transcript.transcript_id;
    """
    cur.execute(create_view_sql)
    write_conn.commit()

create_remark_transcript_context_view("remark_transcript_context", "transcript", "context", "remark")